In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import re
import warnings

warnings.filterwarnings('ignore')

In [ ]:
def scrape(year, months):

    year_data = []

    base_game_url = f'https://www.basketball-reference.com/leagues/NBA_{year}_games-MONTH.html'
    urls = [base_game_url.replace('MONTH', month) for month in months]
    base_url = 'https://www.basketball-reference.com'

    for url in urls:
        response = requests.get(url)
        response.encoding = 'utf-8'
        response.raise_for_status()

        bigsoup = BeautifulSoup(response.text, 'html.parser')

        links = [base_url + a['href'] for a in bigsoup.find_all('a', string="Box Score") if 'href' in a.attrs]

        for link in links:
            # print(f"Scraping: {link}")
            bs = requests.get(link)
            bs.encoding = 'utf-8'
            bs.raise_for_status()
            soup = BeautifulSoup(bs.text, 'html.parser')

            tables = soup.find_all('table', id=re.compile('^box-.*-game-basic$'))

            time.sleep(3)

            data = []
            for table in tables:
                if table:
                    df = pd.read_html(str(table))[0]
                    starters = [df.iloc[i][0] for i in range(5)]
                    reserves = [df.iloc[i][0] for i in range(6,9)]
                    data.append([starters, reserves])

                else:
                    print("Table with the specified ID not found.")

            year_data.append(data)

    print('year scraped')
    return year_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# LOAD GAME DATA

games = pd.read_csv("drive/MyDrive/Colab Notebooks/SlamDunk/games.csv")
games['Date'] = pd.to_datetime(games['Date'])

reg_months = ['october', 'november', 'december', 'january', 'february', 'march', 'april']
months_2021 = ['december', 'january', 'february', 'march', 'april']
months_2020 = ['october', 'november', 'december', 'january', 'february', 'march']

month_dict = {2020: months_2020, 2021: months_2021, 2022: reg_months, 2023: reg_months, 2024: reg_months}


def add_players(games):
    all_players = []

    for year in range(2020, 2025):
        months = month_dict[year]
        players_by_year = scrape(year, months)
        all_players = all_players + players_by_year

    games['players'] = all_players
    games.to_csv('final_test.csv')
    return games

In [ ]:
full = add_players(games)

year scraped
year scraped
year scraped
year scraped
year scraped


In [ ]:
full.to_csv('full.csv')